## 🚀 머신러닝 실습 : 고객 구매 데이터로 성별 예측 모델링 (분류 문제)

- 주어진 데이터는 백화점 고객의 1년 간 구매 데이터입니다.  
- 고객 3,500명에 대한 학습용 데이터(y.csv, X.csv)를 이용하여 성별예측 모형을 만들기.  
- 모델의 성능은 자유롭게 측정하기


### [실습 프로세스]
1. 데이터 불러오기  
2. 데이터 탐색  
3. 데이터 전처리  
4. 학습/테스트 데이터 분리  
5. 모델 선택 및 학습  
6. 예측 및 평가  

### 0. 라이브러리 불러오기

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [50]:
import pandas as pd


from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### 1. 데이터 불러오기

- 데이터를 가져와서 과정을 준비합시다.  

- 인코딩 방식은 'euc-kr' 을 활용하세요.  

- 데이터 출처 : 한국데이터산업진흥원 빅데이터분석기사 실기 공개 예시 문항  

- 독립 변수 데이터셋 : ./data/X.csv  
 
- 종속 변수 데이터셋 : ./data/y.csv  

In [3]:
import os
# 노트북 파일이 있는 폴더로 이동 (예시)
os.chdir(r'C:\githome\hipython_rep')

# 변경 후 확인
print("변경 후:", os.getcwd())

변경 후: C:\githome\hipython_rep


In [4]:
X = pd.read_csv('./data1/X.csv', encoding='euc-kr')
y = pd.read_csv('./data1/y.csv', encoding='euc-kr')
X, y

(      cust_id       총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수  \
 0           0   68282840  11264000  6860000.0      기타   강남점    19  3.894737   
 1           1    2136000   2136000   300000.0     스포츠   잠실점     2  1.500000   
 2           2    3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000   
 3           3   16077620   4935000        NaN      기타   광주점    18  2.444444   
 4           4   29050000  24000000        NaN      보석  본  점     2  1.500000   
 ...       ...        ...       ...        ...     ...   ...   ...       ...   
 3495     3495    3175200   3042900        NaN      골프  본  점     1  2.000000   
 3496     3496   29628600   7200000  6049600.0    시티웨어  부산본점     8  1.625000   
 3497     3497      75000     75000        NaN    주방용품   창원점     1  1.000000   
 3498     3498    1875000   1000000        NaN     화장품  본  점     2  1.000000   
 3499     3499  263101550  34632000  5973000.0      기타  본  점    38  2.421053   
 
         주말방문비율  구매주기  
 0     0.52702

### 2.데이터 탐색하기
- 데이터를 이해할 수 있도록 탐색과정을 수행하기.

- 데이터의 상위 몇 개 행을 출력하여 전체 구조를 미리 확인하기.

In [5]:
X.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [6]:
y.head()

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0


#### 데이터의 요약 정보나 통계 정보를 출력해 변수들의 유형과 분포 확인하기

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [8]:
X.describe()

,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기
count,3500.000000,3.500000e+03,3.500000e+03,1.205000e+03,3500.000000,3500.000000,3500.000000,3500.000000
mean,1749.500000,9.191925e+07,1.966424e+07,2.407822e+07,19.253714,2.834963,0.307246,20.958286
std,1010.507298,1.635065e+08,3.199235e+07,4.746453e+07,27.174942,1.912368,0.289752,24.748682
min,0.000000,-5.242152e+07,-2.992000e+06,5.600000e+03,1.000000,1.000000,0.000000,0.000000
25%,874.750000,4.747050e+06,2.875000e+06,2.259000e+06,2.000000,1.666667,0.027291,4.000000
50%,1749.500000,2.822270e+07,9.837000e+06,7.392000e+06,8.000000,2.333333,0.256410,13.000000
75%,2624.250000,1.065079e+08,2.296250e+07,2.412000e+07,25.000000,3.375000,0.448980,28.000000
max,3499.000000,2.323180e+09,7.066290e+08,5.637530e+08,285.000000,22.083333,1.000000,166.000000


In [9]:
X.isna().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [10]:
X['주구매상품'].head()

0        기타
1       스포츠
2    남성 캐주얼
3        기타
4        보석
Name: 주구매상품, dtype: object

In [10]:
mainproduct_refund=X.groupby('주구매상품')['환불금액'].mean()
mainproduct_refund

주구매상품
가공식품      1.979173e+07
가구        1.011060e+07
건강식품      1.277410e+07
골프        2.146390e+07
구두        3.320762e+06
기타        1.719766e+07
남성 캐주얼    6.450662e+06
남성 트랜디    2.512380e+07
남성정장      4.507000e+06
농산물       2.263326e+07
대형가전      9.732867e+07
디자이너      7.049435e+07
란제리/내의    2.240960e+07
명품        3.892411e+07
모피/피혁     5.531688e+06
보석                 NaN
생활잡화      6.528000e+06
섬유잡화      6.353624e+06
셔츠        3.964160e+06
소형가전               NaN
수산품       8.470131e+06
스포츠       9.834013e+06
시티웨어      1.811968e+07
식기        4.255000e+07
아동        3.021707e+07
악기                 NaN
액세서리      6.888000e+06
육류        1.163039e+07
일용잡화      9.641162e+06
젓갈/반찬     1.470748e+07
주류        5.600000e+03
주방가전      9.446430e+07
주방용품      6.425075e+06
차/커피      6.920356e+06
축산가공      1.087173e+07
침구/수예     3.175245e+07
캐주얼       2.077980e+07
커리어       2.447445e+07
통신/컴퓨터             NaN
트래디셔널     6.542200e+06
피혁잡화      3.356925e+06
화장품       2.198165e+07
Name: 환불금액, dtype: float64

In [11]:
y.value_counts() #성별 분포 확인

cust_id  gender
3499     0         1
0        0         1
1        0         1
2        1         1
3        1         1
                  ..
16       1         1
15       1         1
14       0         1
13       0         1
12       1         1
Name: count, Length: 3500, dtype: int64

## 3.데이터 전처리

- 전처리 과정을 통해서 머신러닝에 사용할 수 있는 형태의 데이터 준비하기  

- 필요한 라이브러리 불러오기   
 \- 인코딩 : LabelEncoder  
 \- 데이터 표준화 : StandardScaler

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

- 단순히 1부터의 숫자를 부여한 'cust_id'를 수치형 변수로 받아들이면  
  결과가 왜곡될 수 있으니 컬럼을 제거합니다.

In [13]:
X=X.drop('cust_id',axis=1)
y=y.drop('cust_id',axis=1)

- 데이터에 결측치가 있는지 확인하기

In [14]:
X.isnull().sum()

총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [15]:
y.isnull().sum()

gender    0
dtype: int64

- 결측치를 0으로 대체하여 모델 학습에 지장이 없도록 하겠습니다

In [16]:
X['환불금액']=X['환불금액'].fillna(0)

- 문자형 범주 데이터를 숫자로 바꾸기 위한 인코딩을 수행합니다.

In [17]:
from sklearn.preprocessing import LabelEncoder

le1=LabelEncoder()
le2=LabelEncoder()
X['주구매상품']=le1.fit_transform(X['주구매상품'])
X['주구매지점']=le2.fit_transform(X['주구매지점'])

- 인코딩 확인

In [18]:
X.head(5)

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,6860000.0,5,0,19,3.894737,0.527027,17
1,2136000,2136000,300000.0,21,19,2,1.500000,0.000000,1
2,3197000,1639000,0.0,6,1,2,2.000000,0.000000,1
3,16077620,4935000,0.0,5,2,18,2.444444,0.318182,16
4,29050000,24000000,0.0,15,8,2,1.500000,0.000000,85


### 각 데이터에 표준화를 적용하여 데이터의 스케일(크기 차이)을 맞춰줍니다.
- 평균을0, 표준편차를 1로 맞춰서 정규 분포 형태로 변환.

In [19]:
y=y['gender']

In [20]:
from sklearn.model_selection import train_test_split
y
X_train,X_test,y_train,y_test=train_test_split(
                        X[['주구매상품','주구매지점','주말방문비율','구매주기']]
                        ,y
                        ,test_size=0.2
                        ,random_state=42)

In [21]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

## 4. 모델링-LogisticRegression
- 본격적으로 모델을 선언하고 학습시킵니다

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

- 모듈 선언하여 객체와 시킵니다

In [23]:
lr_clf=LogisticRegression()

- 모델을 학습 데이터에 맞춰 학습시킵니다

In [24]:
lr_clf.fit(X_train_scaled, y_train)

LogisticRegression()

## 5. 예측 성능 확인해보기 - LogisticRegression
- 학습된 모델로 테스트 데이터에 대한 예측 수행하기

In [25]:
lr_pred=lr_clf.predict(X_test_scaled)

- 학습시킨 모델의 성능 알아보기

In [26]:
accuracy_score(y_test, lr_pred)

0.6085714285714285

- 각 평가지표로 모델의 성늘을 수치화하여 성능 확인하기  
(정확도, 정밀도, 재현율, f1, confusion_matrix)

In [27]:
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
print(confusion_matrix(y_test, lr_pred))
print(classification_report(y_test, lr_pred))

[[421   6]
 [268   5]]
              precision    recall  f1-score   support

           0       0.61      0.99      0.75       427
           1       0.45      0.02      0.04       273

    accuracy                           0.61       700
   macro avg       0.53      0.50      0.39       700
weighted avg       0.55      0.61      0.47       700



## 4-1. 모델링 - DecisionTreeClassifier

- 모델 선언 후 학습

In [28]:
from sklearn.tree import DecisionTreeClassifier

- 모델 선언후 객체화

In [29]:
dt_clf=DecisionTreeClassifier()

- 모델을 학습 데이터에 맞춰 학습시키기

In [30]:
dt_clf.fit(X_train_scaled, y_train)

DecisionTreeClassifier()

## 5-1. 예측 성능 확인해보기- DecisionTreeClassifier

- 학습된 모델로 테스트 데이터에 대한 예측 수행하기

In [31]:
dt_pred=dt_clf.predict(X_test_scaled)

- 학습시킨 모델의 성능 알아보기  
\- 각 평가지표로 모델의 성능을 수치화하여 확인하기  
   (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [32]:
accuracy_score(y_test, dt_pred)

0.5642857142857143

In [33]:
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
print(confusion_matrix(y_test, lr_pred))
print(classification_report(y_test, lr_pred))

[[421   6]
 [268   5]]
              precision    recall  f1-score   support

           0       0.61      0.99      0.75       427
           1       0.45      0.02      0.04       273

    accuracy                           0.61       700
   macro avg       0.53      0.50      0.39       700
weighted avg       0.55      0.61      0.47       700



In [34]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth' : [10,20,30], 'min_samples_split' : [5,15,25]}
grid_dtree = GridSearchCV(dt_clf, param_grid=params, cv=3, refit=True)
grid_dtree.fit(X_train_scaled, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_split': [5, 15, 25]})

In [35]:
b_model=grid_dtree.best_estimator_
pred=b_model.predict(X_test_scaled)
accuracy_score(y_test,pred)

0.6228571428571429

## 4-2.모델링 - RandomForestClassifier

In [36]:
from sklearn.ensemble import RandomForestClassifier

- 모델 선언하여 객체화 시키기

In [37]:
rf_clf = RandomForestClassifier(random_state=42, max_depth=20)

- 모델을 학습 데이터에 맞춰 학습시키기

In [38]:
rf_clf.fit(X_train_scaled,y_train)

RandomForestClassifier(max_depth=20, random_state=42)

## 5-2. 예측 성능 확인해보기 - RandomForestClassifier

- 학습된 모델로 테스트 데이터에 대한 예측 수행하기

In [39]:
rf_pred=rf_clf.predict(X_test_scaled)

- 학습시킨 모델의 성능 알아보기

In [40]:
accuracy_score(y_test,rf_pred)

0.6314285714285715

- 각 평가지표로 모델의 성능을 수치화하여 확인하기  
\- (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [41]:
print(confusion_matrix(y_test, rf_pred))
print(classification_report(y_test, rf_pred))

[[328  99]
 [159 114]]
              precision    recall  f1-score   support

           0       0.67      0.77      0.72       427
           1       0.54      0.42      0.47       273

    accuracy                           0.63       700
   macro avg       0.60      0.59      0.59       700
weighted avg       0.62      0.63      0.62       700



In [42]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth' : [10,20,30], 'min_samples_split' : [5,15,25]}
grid_dtree = GridSearchCV(rf_clf, param_grid=params, cv=3, refit=True)
grid_dtree.fit(X_train_scaled, y_train)

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(max_depth=20, random_state=42),
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_split': [5, 15, 25]})

In [43]:
b_model=grid_dtree.best_estimator_
pred=b_model.predict(X_test_scaled)
accuracy_score(y_test,pred)

0.6428571428571429

## 4-3. 모델링- XGBoost

In [44]:
import xgboost
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping 

- 모델 선언 후 객체화

In [45]:
xgb = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3, use_label_encoder=False)

- 모델을 학습 데이터에 맞춰 학습시키기

In [46]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)  # Series → 1D array
y_test_encoded = le.transform(y_test)

In [47]:
evals = [(X_test_scaled, y_test_encoded)]
xgb.fit(X_train_scaled, y_train_encoded, early_stopping_rounds=40, 
        eval_set=evals,verbose=True)

[0]	validation_0-logloss:0.68279
[1]	validation_0-logloss:0.67466
[2]	validation_0-logloss:0.66847
[3]	validation_0-logloss:0.66368
[4]	validation_0-logloss:0.65934
[5]	validation_0-logloss:0.65561
[6]	validation_0-logloss:0.65285
[7]	validation_0-logloss:0.64956
[8]	validation_0-logloss:0.64705
[9]	validation_0-logloss:0.64534
[10]	validation_0-logloss:0.64374
[11]	validation_0-logloss:0.64103
[12]	validation_0-logloss:0.63941
[13]	validation_0-logloss:0.63795
[14]	validation_0-logloss:0.63682
[15]	validation_0-logloss:0.63551
[16]	validation_0-logloss:0.63470
[17]	validation_0-logloss:0.63359
[18]	validation_0-logloss:0.63345
[19]	validation_0-logloss:0.63249
[20]	validation_0-logloss:0.63090
[21]	validation_0-logloss:0.63025
[22]	validation_0-logloss:0.62988
[23]	validation_0-logloss:0.62940
[24]	validation_0-logloss:0.62777
[25]	validation_0-logloss:0.62730
[26]	validation_0-logloss:0.62699
[27]	validation_0-logloss:0.62615
[28]	validation_0-logloss:0.62599
[29]	validation_0-loglos

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=400,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

## 5-3. 예측 성능 확인해보기 - XGBoost
- 학습된 모델로 테스트 데이터에 대한 예측 수행하기

In [52]:
xgb_pred=xgb.predict(X_test_scaled)

- 학습시킨 모델의 성능 알아보기

In [53]:
accuracy_score(y_test,xgb_pred)

0.6528571428571428

- 각 평가지표로 모델의 성능을 수치화하여 확인하기  
 \-  (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [54]:
print(confusion_matrix(y_test, xgb_pred))
print(classification_report(y_test, xgb_pred))

[[365  62]
 [181  92]]
              precision    recall  f1-score   support

           0       0.67      0.85      0.75       427
           1       0.60      0.34      0.43       273

    accuracy                           0.65       700
   macro avg       0.63      0.60      0.59       700
weighted avg       0.64      0.65      0.63       700



#### 하이퍼파라미터 후보 목록 정의

In [55]:
param_grid = {
    'n_estimators': [100, 200, 400],      # 트리 개수
    'learning_rate': [0.01, 0.1, 0.2],    # 학습률
    'max_depth': [3, 5, 7],               # 트리 최대 깊이
    'subsample': [0.8, 1.0],              # 각 트리 학습에 사용할 샘플 비율
    'colsample_bytree': [0.8, 1.0]        # 각 트리 학습에 사용할 특성 비율
}


#### GridSearchCV로 튜닝 준비 

In [56]:
xgb_clf = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',     # 최신 XGBoost에서는 eval_metric 지정 필수
    random_state=42
)

grid_cv = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,                     # 3-fold cross validation
    verbose=2,
    n_jobs=-1
)

#### 모델 학습(튜닝 진행)

In [57]:
grid_cv.fit(X_train_scaled, y_train_encoded)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     m...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=42, reg_alpha=None,
                                     reg_lambda=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 400],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy', verbose=2)

#### 최적 파라미터 및 성능확인

In [58]:
print("최적 하이퍼파라미터:", grid_cv.best_params_)
print("최고 검증 정확도:", grid_cv.best_score_)

최적 하이퍼파라미터: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 400, 'subsample': 1.0}
최고 검증 정확도: 0.6553529747929246


#### 최적 모델로 예측 및 평가

In [59]:
best_xgb = grid_cv.best_estimator_
xgb_pred = best_xgb.predict(X_test_scaled)
acc = accuracy_score(y_test_encoded, xgb_pred)
print("테스트셋 정확도:", acc)

테스트셋 정확도: 0.6428571428571429
